<a href="https://colab.research.google.com/github/SupriyaUpadhyaya/HCNLP-Text2Sql-Project/blob/main/SQLAssist_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes
!pip install huggingface_hub[hf_transfer]

Imports

In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

Load the base model

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep


In [ ]:
text_to_sql_tmpl_str = """\
### Instruction:\n{system_message}{user_message}\n\n### Response:\n{response}"""

text_to_sql_inference_tmpl_str = """\
### Instruction:\n{system_message}{user_message}\n\n### Response:\n"""


def _generate_prompt_sql(input, context, dialect="sqlite", output=""):
    system_message = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

    """
    user_message = f"""### Dialect:
{dialect}

### Input:
{input}

### Context:
{context}

### Response:
"""
    if output:
        return text_to_sql_tmpl_str.format(
            system_message=system_message,
            user_message=user_message,
            response=output,
        )
    else:
        return text_to_sql_inference_tmpl_str.format(
            system_message=system_message, user_message=user_message
        )


def generate_prompt(data_point):
    full_prompt = _generate_prompt_sql(
        data_point["instruction"],
        data_point["input"],
        dialect="sqlite",
        output=data_point["response"],
    )
    EOS_TOKEN = tokenizer.eos_token
    return full_prompt + EOS_TOKEN

In [ ]:
# Define the dataset for fine-tuning
dataset_id = "Clinton/Text-to-sql-v1"

data = load_dataset(dataset_id, split="train")
df = data.to_pandas()

Generating train split:   0%|          | 0/262208 [00:00<?, ? examples/s]

In [ ]:
# Apply the chat_template_for_training function to each row in the
# dataframe and store the result in a new "text" column.
df["text"] = df.apply(lambda x: generate_prompt(x), axis=1)

# Convert the dataframe back to a Dataset object.
formatted_data = Dataset.from_pandas(df)

In [ ]:
df["text"][0]

'### Instruction:\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.\n\nYou must output the SQL query that answers the question.\n\n    ### Dialect:\nsqlite\n\n### Input:\nName the home team for carlton away team\n\n### Context:\nCREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n)\n\n### Response:\n\n\n### Response:\nSELECT home_team FROM table_name_77 WHERE away_team = "carlton"<|end_of_text|>'

<a name="Train"></a>
### Train the model

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/262208 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.594 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 262,208 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.035300
2,1.557900
3,2.147200
4,1.637300
5,1.114700
6,1.481800
7,1.686200
8,1.209800
9,0.924600
10,0.929900


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

329.7482 seconds used for training.
5.5 minutes used for training.
Peak reserved memory = 6.311 GB.
Peak reserved memory for training = 0.717 GB.
Peak reserved memory % of max memory = 42.792 %.
Peak reserved memory for training % of max memory = 4.862 %.


<a name="Save"></a>
### Saving finetuned models


In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:
model.save_pretrained_merged("model_16", tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.19 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:22<00:00,  2.58s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model_16/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model_16/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model_16/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model_16/pytorch_model-00004-of-00004.bin...
Done.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

To speed up the upload, set the following variable

In [ ]:
%env HF_HUB_ENABLE_HF_TRANSFER=1

env: HF_HUB_ENABLE_HF_TRANSFER=1


In [ ]:
# model.push_to_hub_merged("basavaraj/text2sql-Llama3-8b", tokenizer, save_method = "merged_16bit", token = "<My_Token>")
!huggingface-cli upload basavaraj/text2sql-Llama3-8b model_16 .

  0% 0/4 [00:00<?, ?it/s]
pytorch_model-00001-of-00004.bin:   0% 0.00/4.98G [00:00<?, ?B/s]
pytorch_model-00001-of-00004.bin:   0% 16.0M/4.98G [00:12<1:04:16, 1.29MB/s]
pytorch_model-00001-of-00004.bin:   1% 32.0M/4.98G [00:14<32:22, 2.55MB/s]  
pytorch_model-00001-of-00004.bin:   1% 48.0M/4.98G [00:15<20:00, 4.10MB/s]
pytorch_model-00001-of-00004.bin:   1% 64.0M/4.98G [00:15<12:20, 6.63MB/s]
pytorch_model-00001-of-00004.bin:   2% 80.0M/4.98G [00:15<08:04, 10.1MB/s]
pytorch_model-00001-of-00004.bin:   2% 96.0M/4.98G [00:15<05:39, 14.4MB/s]
pytorch_model-00001-of-00004.bin:   3% 128M/4.98G [00:16<03:12, 25.2MB/s] 
pytorch_model-00001-of-00004.bin:   3% 160M/4.98G [00:16<02:07, 37.7MB/s]
pytorch_model-00001-of-00004.bin:   4% 176M/4.98G [00:16<02:10, 36.8MB/s]
pytorch_model-00001-of-00004.bin:   5% 224M/4.98G [00:17<01:13, 64.8MB/s]
pytorch_model-00001-of-00004.bin:   5% 256M/4.98G [00:17<01:01, 77.0MB/s]
pytorch_model-00001-of-00004.bin:   6% 304M/4.98G [00:17<00:43, 108MB/s] 
pytorch_m